<a href="https://colab.research.google.com/github/jnsbrdbr/hopefully_last_thesis/blob/main/92_1000_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

from sklearn.decomposition import PCA
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **importing data**

In [3]:
import tables
tables.file._open_files.close_all()

In [4]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tdata.h5')

adj=store['adj']
store.close()

In [5]:
with pd.HDFStore('/content/drive/MyDrive/1000/TCFdf.h5',  mode='r') as newstore:
    CF = newstore.select('CFdf')

In [6]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TTFdf.h5')
TF=store['TFdf']
store.close()

In [7]:
with pd.HDFStore('/content/drive/MyDrive/1000/TjaccardsimilaritydfR.h5',  mode='r') as newstore:
    jaccardsimilarity = newstore.select('jaccardsimilaritydfR')

In [8]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TcosinesimilaritydfR.h5')
cosine=store['cosine_similaritydfR']

In [9]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TL1normdf.h5')
L1_normdf=store['L1_normdf']
store.close()

In [10]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tsum.h5')
wsumdf=store['wsumdf']
store.close()

In [11]:
with pd.HDFStore('/content/drive/MyDrive/1000/Tmul.h5',  mode='r') as newstore:
    wmuldf = newstore.select('wmuldf')

In [12]:
with pd.HDFStore('/content/drive/MyDrive/1000/TLdata.h5',  mode='r') as newstore:
    adjl = newstore.select('adjl')

# **implementation of PCA**

In [13]:
pca = PCA(n_components=10)
CF_pca = pca.fit_transform(CF)
CF_pca_df=pd.DataFrame(CF_pca)

In [14]:
CF_pca_df

,0,1,2,3,4,5,6,7,8,9
0,-1.715972,-3.099936,-0.041249,-2.772767e-14,4.332345e-14,0.077295,1.691428,3.341114,-7.723126e-15,1.346899e-14
1,-1.715972,-3.099936,-0.041249,1.466600e-13,2.316440e-14,0.077295,1.691428,3.341114,6.940363e-14,-8.596380e-14
2,-1.715972,-3.099936,-0.041249,7.395959e-14,2.474722e-14,0.077295,1.691428,3.341114,-3.695459e-14,8.705053e-14
3,-1.715972,-3.099936,-0.041249,-4.481138e-14,3.516572e-14,0.077295,1.691428,3.341114,1.210173e-13,-2.066566e-13
4,-1.715972,-3.099936,-0.041249,-1.149663e-14,2.689488e-14,0.077295,1.691428,3.341114,-1.479650e-13,2.326630e-13
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,-5.411471e-13,2.859262e-14,8.797502,-0.661164,-0.063010,-5.418220e-02,-1.896785e-04
996,-2.114893,-4.572474,-0.754679,-5.375491e-13,2.843942e-14,8.797502,-0.661164,-0.063010,-2.275366e-02,4.725734e-02
997,-2.114893,-4.572474,-0.754679,-5.376020e-13,2.778318e-14,8.797502,-0.661164,-0.063010,-2.662274e-02,-4.298936e-02
998,-2.114893,-4.572474,-0.754679,-5.381695e-13,2.793383e-14,8.797502,-0.661164,-0.063010,1.507282e-02,-1.728412e-02


In [15]:
pca = PCA(n_components=10)
TF_pca = pca.fit_transform(TF)
TF_pca_df=pd.DataFrame(TF_pca)

In [16]:
TF_pca_df.rename(columns = {0:10,1:11,2:12,3:13,4:14,5:15,6:16,7:17,8:18,9:19}, inplace = True)

In [17]:
TF_pca_df

,10,11,12,13,14,15,16,17,18,19
0,-26.167339,0.109748,0.187462,-0.926807,0.001555,1.121668e-11,-1.052847e-11,-0.002291,-1.817205,-0.021183
1,-26.167339,0.109748,0.187462,-0.926807,0.001555,-5.736279e-11,5.420585e-11,-0.002291,-1.817205,-0.021183
2,-26.167339,0.109748,0.187462,-0.926807,0.001555,1.540322e-11,-1.485367e-11,-0.002291,-1.817205,-0.021183
3,-26.167339,0.109748,0.187462,-0.926807,0.001555,2.736459e-11,-2.591391e-11,-0.002291,-1.817205,-0.021183
4,1393.385962,-11.935691,-29.759005,-0.507027,-0.476180,2.519848e-13,-2.368198e-13,0.884093,-1.749524,-0.183266
...,...,...,...,...,...,...,...,...,...,...
995,-29.238191,0.128642,0.210675,-1.647476,0.001097,1.470064e-13,-3.260901e-13,-0.000867,-4.540979,-0.763937
996,-29.238191,0.128642,0.210675,-1.647476,0.001097,1.408862e-13,-3.201265e-13,-0.000867,-4.540979,-0.763937
997,-29.238191,0.128642,0.210675,-1.647476,0.001097,1.445492e-13,-3.235842e-13,-0.000867,-4.540979,-0.763937
998,-29.238191,0.128642,0.210675,-1.647476,0.001097,1.382352e-13,-3.176165e-13,-0.000867,-4.540979,-0.763937


In [18]:
pca = PCA(n_components=10)
jaccard_similarity_pca = pca.fit_transform(jaccardsimilarity)
jaccard_similarity_pca_df=pd.DataFrame(jaccard_similarity_pca)

In [19]:
jaccard_similarity_pca_df.rename(columns = {0:20,1:21,2:22,3:23,4:24,5:25,6:26,7:27,8:28,9:29}, inplace = True)

In [20]:
jaccard_similarity_pca_df

,20,21,22,23,24,25,26,27,28,29
0,-1.715972,-3.099936,-0.041249,-2.601953e-14,5.618941e-14,0.077295,1.691428,3.341114,1.293005e-14,-1.498502e-13
1,-1.715972,-3.099936,-0.041249,-1.136234e-13,-4.877982e-15,0.077295,1.691428,3.341114,-9.060674e-14,-6.396161e-14
2,-1.715972,-3.099936,-0.041249,-1.394834e-13,4.513124e-14,0.077295,1.691428,3.341114,-1.961015e-13,-4.118260e-14
3,-1.715972,-3.099936,-0.041249,-4.975901e-14,3.680866e-14,0.077295,1.691428,3.341114,-1.057063e-14,3.484086e-14
4,-1.715972,-3.099936,-0.041249,-6.334567e-14,3.996291e-14,0.077295,1.691428,3.341114,-2.114298e-14,2.153271e-14
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,-3.937646e-13,1.929509e-13,8.797502,-0.661164,-0.063010,3.729270e-03,-4.457939e-02
996,-2.114893,-4.572474,-0.754679,-3.936648e-13,1.935889e-13,8.797502,-0.661164,-0.063010,3.264255e-02,1.818962e-02
997,-2.114893,-4.572474,-0.754679,-3.959402e-13,1.935858e-13,8.797502,-0.661164,-0.063010,-9.851666e-03,1.847036e-02
998,-2.114893,-4.572474,-0.754679,-3.949887e-13,1.935041e-13,8.797502,-0.661164,-0.063010,2.230000e-02,4.536707e-02


In [21]:
pca = PCA(n_components=10)
cosine_similarity_pca = pca.fit_transform(cosine)
cosine_similarity_pca_df=pd.DataFrame(cosine_similarity_pca)

In [22]:
cosine_similarity_pca_df.rename(columns = {0:30,1:31,2:32,3:33,4:34,5:35,6:36,7:37,8:38,9:39}, inplace = True)

In [23]:
cosine_similarity_pca_df

,30,31,32,33,34,35,36,37,38,39
0,-1.715972,-3.099936,-0.041249,-1.438854e-14,3.791477e-15,0.077295,1.691428,3.341114,7.765440e-14,-4.665951e-14
1,-1.715972,-3.099936,-0.041249,4.376833e-16,-2.921238e-14,0.077295,1.691428,3.341114,1.289185e-14,-2.625089e-14
2,-1.715972,-3.099936,-0.041249,-1.715742e-13,1.885603e-13,0.077295,1.691428,3.341114,-5.099649e-14,1.655186e-13
3,-1.715972,-3.099936,-0.041249,-1.323491e-13,-1.138568e-13,0.077295,1.691428,3.341114,-3.021833e-17,-6.204712e-14
4,-1.715972,-3.099936,-0.041249,2.580249e-14,-1.258120e-14,0.077295,1.691428,3.341114,-3.768638e-14,-2.662542e-14
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,2.146372e-12,-3.676991e-13,8.797502,-0.661164,-0.063010,-8.201325e-03,-5.852546e-02
996,-2.114893,-4.572474,-0.754679,2.146181e-12,-3.673655e-13,8.797502,-0.661164,-0.063010,2.363818e-02,-5.643994e-02
997,-2.114893,-4.572474,-0.754679,2.149061e-12,-3.675501e-13,8.797502,-0.661164,-0.063010,1.143620e-02,-2.289446e-02
998,-2.114893,-4.572474,-0.754679,2.148535e-12,-3.673440e-13,8.797502,-0.661164,-0.063010,1.696511e-02,1.012032e-02


In [24]:
pca = PCA(n_components=10)
L1_normdf_similarity_pca = pca.fit_transform(L1_normdf)
L1_normdf_similarity_pca_df=pd.DataFrame(L1_normdf_similarity_pca)

In [25]:
L1_normdf_similarity_pca_df.rename(columns = {0:40,1:41,2:42,3:43,4:44,5:45,6:46,7:47,8:48,9:49}, inplace = True)

In [26]:
L1_normdf_similarity_pca_df

,40,41,42,43,44,45,46,47,48,49
0,-1.715972,-3.099936,-0.041249,-1.554703e-14,-2.355436e-14,0.077295,1.691428,3.341114,8.437575e-15,4.726067e-14
1,-1.715972,-3.099936,-0.041249,-1.746815e-14,4.727301e-14,0.077295,1.691428,3.341114,5.248802e-14,5.591737e-14
2,-1.715972,-3.099936,-0.041249,-1.234652e-13,1.944209e-13,0.077295,1.691428,3.341114,-9.490977e-14,-3.031738e-13
3,-1.715972,-3.099936,-0.041249,-4.199757e-14,-1.117551e-14,0.077295,1.691428,3.341114,-1.677366e-13,-2.720006e-13
4,-1.715972,-3.099936,-0.041249,-4.488508e-14,1.323467e-14,0.077295,1.691428,3.341114,1.087031e-13,1.983893e-13
...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,2.286955e-13,-3.561509e-13,8.797502,-0.661164,-0.063010,-6.077050e-03,2.966413e-02
996,-2.114893,-4.572474,-0.754679,2.285018e-13,-3.560924e-13,8.797502,-0.661164,-0.063010,-3.637081e-02,-1.466903e-02
997,-2.114893,-4.572474,-0.754679,2.291475e-13,-3.558335e-13,8.797502,-0.661164,-0.063010,5.344601e-03,3.042290e-03
998,-2.114893,-4.572474,-0.754679,2.284697e-13,-3.563128e-13,8.797502,-0.661164,-0.063010,3.645226e-04,-1.694720e-02


In [27]:
pca = PCA(n_components=10)
wsumdf_similarity_pca = pca.fit_transform(wsumdf)
wsumdf_similarity_pca_df=pd.DataFrame(wsumdf_similarity_pca)

In [28]:
wsumdf_similarity_pca_df.rename(columns = {0:50,1:51,2:52,3:53,4:54,5:55,6:56,7:57,8:58,9:59}, inplace = True)

In [29]:
wsumdf_similarity_pca_df

,50,51,52,53,54,55,56,57,58,59
0,-0.158046,-8.265557e-03,6.084363e-02,2.985141e-02,-3.735018e-02,1.763228e-02,2.582624e-02,1.890848e-04,1.365839e-01,2.632791e-02
1,-0.158046,4.362438e-02,3.748978e-02,4.307308e-02,3.297469e-02,1.762136e-02,1.054154e-02,8.556521e-02,1.867480e-02,6.838267e-04
2,-0.158046,5.778182e-02,2.675783e-02,9.542582e-03,7.178640e-02,7.794557e-02,1.202657e-01,-2.823838e-02,1.966042e-03,-2.999012e-02
3,-0.158046,-3.160237e-02,2.912844e-03,-6.683623e-02,-6.576034e-02,1.851888e-02,-5.000451e-02,3.766362e-03,-8.744048e-03,8.348055e-02
4,17.554873,-2.985391e-12,-4.058515e-15,1.100844e-12,9.649107e-13,1.102085e-12,1.560483e-12,-1.724964e-12,1.342742e-12,-1.914462e-12
...,...,...,...,...,...,...,...,...,...,...
995,-0.158046,5.156906e-02,1.100328e-02,3.348356e-02,8.564451e-02,2.992314e-02,3.082811e-02,4.685006e-02,1.543280e-02,3.370607e-02
996,-0.158046,-1.710036e-02,-6.219311e-02,-7.621148e-03,2.171903e-02,-1.798500e-02,1.004791e-01,7.422989e-02,-6.907469e-02,4.368369e-02
997,-0.158046,6.795922e-02,1.528332e-02,7.447697e-02,-4.726156e-02,9.910969e-02,1.319143e-02,4.829530e-02,-6.207805e-02,-5.930574e-02
998,-0.158046,7.082751e-02,-5.576869e-02,-3.219825e-02,6.177334e-02,-6.231407e-03,1.026333e-02,-1.234386e-01,3.257567e-02,3.142436e-02


In [30]:
pca = PCA(n_components=10)
wmuldf_similarity_pca = pca.fit_transform(wmuldf)
wmuldf_similarity_pca_df=pd.DataFrame(wmuldf_similarity_pca)

In [31]:
wmuldf_similarity_pca_df.rename(columns = {0:60,1:61,2:62,3:63,4:64,5:65,6:66,7:67,8:68,9:69}, inplace = True)

In [32]:
wmuldf_similarity_pca_df

,60,61,62,63,64,65,66,67,68,69
0,-0.111385,8.552268e-03,-7.904882e-03,-1.212833e-03,-1.659093e-03,1.148052e-02,1.006673e-03,-3.733283e-03,-6.138676e-03,-1.311463e-03
1,-0.111385,-3.740869e-03,7.979555e-03,-2.192725e-02,1.557035e-02,-9.506592e-03,-1.656848e-02,-2.315918e-02,3.512108e-03,-8.894766e-03
2,-0.111385,-2.983303e-02,-1.798253e-03,-5.303236e-03,3.443212e-03,5.501828e-03,1.251424e-02,1.744383e-02,-1.360941e-02,1.550322e-02
3,-0.111385,-3.381541e-04,-1.686177e-02,2.876218e-03,-1.164208e-02,4.846784e-03,-1.108571e-02,-5.616712e-03,-8.548495e-03,-1.204656e-02
4,12.371234,7.341793e-15,-7.293987e-15,-7.855589e-15,-4.889254e-15,1.341662e-14,-5.313069e-15,-3.317777e-15,-5.544234e-15,2.698971e-15
...,...,...,...,...,...,...,...,...,...,...
995,-0.111385,-4.855958e-02,-2.616962e-03,6.899072e-03,-4.836771e-03,3.054156e-03,6.665845e-03,7.002829e-04,8.451918e-03,-9.629763e-03
996,-0.111385,-7.965602e-03,1.382855e-02,1.030486e-02,4.727405e-04,-3.912152e-03,-2.744242e-02,-8.723051e-03,-8.249622e-03,1.890977e-02
997,-0.111385,4.801682e-03,2.631254e-02,-1.099122e-03,-1.685858e-02,-3.284569e-02,1.066669e-02,-9.583355e-04,4.785683e-02,-7.653977e-03
998,-0.111385,1.853990e-02,-1.137538e-02,-3.079020e-02,-1.654584e-03,1.811239e-02,1.087194e-02,1.212881e-02,7.125102e-04,-1.037818e-02


In [33]:
y=adjl.label
y_arr=np.array(y)
y_df=pd.DataFrame(y_arr)

In [34]:
y_df.rename(columns = {0:'label'}, inplace = True)

In [35]:
#concat
frames = [CF_pca_df, TF_pca_df, jaccard_similarity_pca_df,cosine_similarity_pca_df,
          L1_normdf_similarity_pca_df,wsumdf_similarity_pca_df,wmuldf_similarity_pca_df,y_df]
result = pd.concat(frames,axis=1)

In [36]:
result

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,label
0,-1.715972,-3.099936,-0.041249,-2.772767e-14,4.332345e-14,0.077295,1.691428,3.341114,-7.723126e-15,1.346899e-14,...,8.552268e-03,-7.904882e-03,-1.212833e-03,-1.659093e-03,1.148052e-02,1.006673e-03,-3.733283e-03,-6.138676e-03,-1.311463e-03,1
1,-1.715972,-3.099936,-0.041249,1.466600e-13,2.316440e-14,0.077295,1.691428,3.341114,6.940363e-14,-8.596380e-14,...,-3.740869e-03,7.979555e-03,-2.192725e-02,1.557035e-02,-9.506592e-03,-1.656848e-02,-2.315918e-02,3.512108e-03,-8.894766e-03,1
2,-1.715972,-3.099936,-0.041249,7.395959e-14,2.474722e-14,0.077295,1.691428,3.341114,-3.695459e-14,8.705053e-14,...,-2.983303e-02,-1.798253e-03,-5.303236e-03,3.443212e-03,5.501828e-03,1.251424e-02,1.744383e-02,-1.360941e-02,1.550322e-02,1
3,-1.715972,-3.099936,-0.041249,-4.481138e-14,3.516572e-14,0.077295,1.691428,3.341114,1.210173e-13,-2.066566e-13,...,-3.381541e-04,-1.686177e-02,2.876218e-03,-1.164208e-02,4.846784e-03,-1.108571e-02,-5.616712e-03,-8.548495e-03,-1.204656e-02,1
4,-1.715972,-3.099936,-0.041249,-1.149663e-14,2.689488e-14,0.077295,1.691428,3.341114,-1.479650e-13,2.326630e-13,...,7.341793e-15,-7.293987e-15,-7.855589e-15,-4.889254e-15,1.341662e-14,-5.313069e-15,-3.317777e-15,-5.544234e-15,2.698971e-15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,-5.411471e-13,2.859262e-14,8.797502,-0.661164,-0.063010,-5.418220e-02,-1.896785e-04,...,-4.855958e-02,-2.616962e-03,6.899072e-03,-4.836771e-03,3.054156e-03,6.665845e-03,7.002829e-04,8.451918e-03,-9.629763e-03,1
996,-2.114893,-4.572474,-0.754679,-5.375491e-13,2.843942e-14,8.797502,-0.661164,-0.063010,-2.275366e-02,4.725734e-02,...,-7.965602e-03,1.382855e-02,1.030486e-02,4.727405e-04,-3.912152e-03,-2.744242e-02,-8.723051e-03,-8.249622e-03,1.890977e-02,1
997,-2.114893,-4.572474,-0.754679,-5.376020e-13,2.778318e-14,8.797502,-0.661164,-0.063010,-2.662274e-02,-4.298936e-02,...,4.801682e-03,2.631254e-02,-1.099122e-03,-1.685858e-02,-3.284569e-02,1.066669e-02,-9.583355e-04,4.785683e-02,-7.653977e-03,1
998,-2.114893,-4.572474,-0.754679,-5.381695e-13,2.793383e-14,8.797502,-0.661164,-0.063010,1.507282e-02,-1.728412e-02,...,1.853990e-02,-1.137538e-02,-3.079020e-02,-1.654584e-03,1.811239e-02,1.087194e-02,1.212881e-02,7.125102e-04,-1.037818e-02,1


In [37]:
with pd.HDFStore('/content/drive/MyDrive/last/last_result_1000',  mode='w') as store:
    store.append('result', result,format='table')